In [36]:
import os
import json
import numpy as np
import pandas as pd

In [37]:
def read_json(path):
    res = {'Magnitude': [], 'PetID': [], 'Score': []}
    for file in os.listdir(path):
        with open(os.path.join(path, file)) as f:
            d = json.load(f)['documentSentiment']
            res['Magnitude'].append(d['magnitude'])
            res['PetID'].append(file.split('.', 1)[0])
            res['Score'].append(d['score'])
    return pd.DataFrame(res)

In [38]:
def breed_counts(train):
    counts = pd.DataFrame(train.groupby('Breed1').size(), columns=['Count']).reset_index()
    joined1 = train.merge(counts, on='Breed1', how='left')
#     joined1['IsPure'] = (joined1['Breed2'] == 0).astype(int)
#     joined2 = joined1.merge(ranks.rename(columns={'Breed1': 'Breed2'}), on='Breed2', how='left') \
#         .drop(columns=['Breed2']) \
#         .rename(columns={'Rank': 'Breed2'})
#     joined2['Breed2'] = joined2.apply(lambda row: 0 if row['IsPure'] else row['Breed2'], axis=1)
#     return joined2.drop(columns=['IsPure']).fillna(MAX_SPECIES + 1)
    joined1['IsPure'] = (np.logical_or(joined1['Breed2'] == 0, joined1['Breed1'] == MAX_SPECIES)).astype(int)
    return joined1.fillna(0)

In [39]:
MAX_SPECIES = 307

In [42]:
train = pd.read_csv('data/train.csv')
sentiment = read_json('data/train_sentiment')
train = train.merge(sentiment, how = 'left', on = 'PetID')
# train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test/test.csv')

train = train.drop(columns = ['Name', 'RescuerID', 'PetID', 'Description'])

In [43]:
train.isna().any()

Type             False
Age              False
Breed1           False
Breed2           False
Gender           False
Color1           False
Color2           False
Color3           False
MaturitySize     False
FurLength        False
Vaccinated       False
Dewormed         False
Sterilized       False
Health           False
Quantity         False
Fee              False
State            False
VideoAmt         False
PhotoAmt         False
AdoptionSpeed    False
Magnitude         True
Score             True
dtype: bool

In [44]:
train.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed', 'Magnitude', 'Score'],
      dtype='object')

In [45]:
for column in train.columns:
    print(f'{column}: {set(train[column])}')

Type: {1, 2}
Age: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 85, 86, 87, 88, 89, 91, 92, 95, 96, 100, 102, 108, 112, 117, 120, 122, 123, 132, 135, 144, 147, 156, 168, 180, 212, 238, 255}
Breed1: {0, 1, 3, 5, 7, 10, 11, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 31, 32, 39, 42, 44, 49, 50, 56, 58, 60, 61, 64, 65, 69, 70, 71, 72, 75, 76, 78, 81, 82, 83, 85, 88, 93, 97, 98, 99, 100, 102, 103, 105, 108, 109, 111, 114, 117, 119, 122, 123, 125, 128, 129, 130, 132, 139, 141, 143, 145, 146, 147, 148, 150, 152, 154, 155, 165, 167, 169, 173, 176, 178, 179, 182, 185, 187, 188, 189, 190, 192, 195, 197, 199, 200, 201, 202, 203, 204, 205, 206, 207, 212, 213, 214, 215, 217, 218, 224, 227, 228, 231, 232, 233, 234, 237, 239, 240, 241, 242, 24

In [46]:
train = breed_counts(train)

In [48]:
train.to_csv('data/clean_train.csv')

In [55]:
train[train['Count'] == 1].count()

Type             33
Age              33
Breed1           33
Breed2           33
Gender           33
Color1           33
Color2           33
Color3           33
MaturitySize     33
FurLength        33
Vaccinated       33
Dewormed         33
Sterilized       33
Health           33
Quantity         33
Fee              33
State            33
VideoAmt         33
PhotoAmt         33
AdoptionSpeed    33
Magnitude        33
Score            33
Count            33
IsPure           33
dtype: int64